In [ ]:
# Import and call the Riak client to connect to your Riak TS node or cluster
# See http://docs.basho.com/riak/ts/latest/developing/python/ for more information
# on how to use the Riak client for Python
from riak import RiakClient

# Note: If you wish Riak TS to return date fields as Python dates
# and not epoch you need to set the transport_options as shown below
client = RiakClient(transport_options={'ts_convert_timestamp': True})

from datetime import datetime
import calendar

table = "WaterMeterData"

In [ ]:
# Select one row of data from our table using the primary key and .ts_get(table,key)
key = [datetime(2016, 7, 6, 12, 0), 'CUSTOMER-0001', 'METER-0001']
data_set = client.ts_get(table, key)
print(data_set.rows[0])

In [ ]:
# Select a range of rows from our table by data (between 6/1/2016 and 7/1/2016)
# We have to convert our dates to Unix Epoch to perform a SQL based query
# See also: http://docs.basho.com/riak/ts/latest/using/querying/ 
start_date = calendar.timegm(datetime.timetuple(datetime(2016, 6, 1, 0, 0))) * 1000
end_date = calendar.timegm(datetime.timetuple(datetime(2016, 7, 1, 1, 0))) * 1000

query = """\
SELECT * FROM WaterMeterData 
WHERE time_stamp >= {} AND time_stamp < {}
""".format(start_date, end_date)

print query

data_set = client.ts_query(table, query)
rowcount = len(data_set.rows)

for row in data_set.rows:
    print(row)
print("Total Rows: {}".format(rowcount))

In [ ]:
# Modify the above query to filter on a field in addition to the primary key fields
query = """\
SELECT * FROM WaterMeterData
WHERE time_stamp >= {} AND time_stamp < {} AND
    fault_condition = True
""".format(start_date, end_date)

data_set = client.ts_query(table, query)
rowcount = len(data_set.rows)

for row in data_set.rows:
    print(row)
print("Total Rows: {}".format(rowcount))

In [ ]:
# The following query demonstrates basic Riak TS aggregate and arithmetic functions
# See: http://docs.basho.com/riak/ts/latest/using/aggregate-functions/
# and: http://docs.basho.com/riak/ts/latest/using/arithmetic-operations/
query = """\
SELECT
    max(total_gallons) - min(total_gallons), avg(water_pressure), avg(gallons_per_hour)
FROM WaterMeterData
WHERE time_stamp >= {} AND time_stamp < {} AND
    meter_id = 'METER-0001'
""".format(start_date, end_date)

data_set = client.ts_query(table, query)
print "Total Gallons: " + str(data_set.rows[0][0])
print "Average Pressure: " + str(data_set.rows[0][1])
print "Avg. Gallons/Hour: " + str(data_set.rows[0][2])